### use the stimuli of study 2

In [25]:
import pandas as pd
import numpy as np
dat = pd.read_csv('final_data_2.csv')
df = pd.DataFrame(dat)



In [26]:



filtered_df = df[(df['test_part'].isin(['cs'])) & (df['skew'].isin(['lr', 'rl', 'ns'])) & (df['Prolific_ID'].isin(['5638e8a444e8c8000ee86a35']))]

# Define the skewness calculation function
def calculate_skewness(probabilities, outcomes):
    # Mean (expected value)
    mu = np.sum(outcomes * probabilities)
    
    # Variance
    sigma_squared = np.sum((outcomes**2) * probabilities) - mu**2
    
    # Third Central Moment
    mu_3 = np.sum(((outcomes - mu)**3) * probabilities)
    
    # Skewness
    skewness = mu_3 / (sigma_squared**(3/2)) if sigma_squared != 0 else 0  # To handle division by zero
    
    return skewness


selected_columns = ['skew', 'P_A1', 'O_A1', 'P_A2', 'O_A2', 'P_B1', 'O_B1', 'P_B2', 'O_B2']
filtered_df = filtered_df[selected_columns]



final_df = filtered_df.assign(
    EVA=lambda x: x['P_A1'] * x['O_A1'] + x['P_A2'] * x['O_A2'],
    EVB=lambda x: x['P_B1'] * x['O_B1'] + x['P_B2'] * x['O_B2'],
    EVD=lambda x: x['EVA'] - x['EVB'],
    SDA=lambda x: np.sqrt(x['P_A1'] * (x['O_A1'] - x['EVA'])**2 + x['P_A2'] * (x['O_A2'] - x['EVA'])**2),
    SDB=lambda x: np.sqrt(x['P_B1'] * (x['O_B1'] - x['EVB'])**2 + x['P_B2'] * (x['O_B2'] - x['EVB'])**2),
    SDD=lambda x: x['SDA'] - x['SDB'],
    skewness_a=lambda x: x.apply(lambda row: calculate_skewness(
        np.array([row['P_A1'], row['P_A2']]), 
        np.array([row['O_A1'], row['O_A2']])
    ), axis=1),
    skewness_b=lambda x: x.apply(lambda row: calculate_skewness(
        np.array([row['P_B1'], row['P_B2']]), 
        np.array([row['O_B1'], row['O_B2']])
    ), axis=1),
    skewness_diff=lambda x: x['skewness_a'] - x['skewness_b']
)



# Bin the 'EVD' column
final_df['evd_bins'] = pd.cut(
    final_df['EVD'], 
    bins=[-float('inf'), -15, -9, 1, 11, 21], 
    labels=["-21 to -19", "-11 to -9", "-1 to 1", "9 to 11", "19 to 21"],
    right=True,
    include_lowest=True
)

# Bin the 'SDD' column
final_df['sdd_bins'] = pd.cut(
    final_df['SDD'], 
    bins=[-float('inf'), 8, 13, 18], 
    labels=["4 to 6", "9 to 11", "14 to 16"],
    right=True,
    include_lowest=True
)

# Sort the DataFrame by 'evd_bins' and then 'sdd_bins'
final_df = final_df.sort_values(by=['evd_bins', 'sdd_bins'])




# Reset the index to remove the existing one and create a new sequential index
final_df = final_df.reset_index(drop=True)

# Add a new 'index' column starting from 1 to 45
final_df['index'] = range(1, 46)

# Move the 'index' column to the beginning of the DataFrame
final_df = final_df[['index'] + [col for col in final_df.columns if col != 'index']]

final_df_new = final_df


In [ ]:

from scipy.optimize import minimize
import random
random.seed(42)  # Set the random seed


# Function to calculate skewness
def calculate_skewness(probabilities, outcomes):
    mu = np.sum(outcomes * probabilities)
    sigma_squared = np.sum((outcomes**2) * probabilities) - mu**2
    sigma = np.sqrt(sigma_squared)
    mu_3 = np.sum(((outcomes - mu)**3) * probabilities)
    skewness = mu_3 / (sigma**3) if sigma != 0 else 0
    return skewness

# Function to calculate EV, SD, and skewness
def lottery_stats(probs, outcomes):
    ev = np.sum(probs * outcomes)
    sd = np.sqrt(np.sum(probs * (outcomes - ev) ** 2))
    skw = calculate_skewness(probs, outcomes)
    return ev, sd, skw

# Generate unique probabilities that sum to 1
def generate_unique_probs(n, low=0.02, high=0.4):
    possible_probs = np.arange(low, high + 0.001, 0.01)
    possible_probs = np.round(possible_probs, 2)
    possible_probs = possible_probs[(possible_probs >= low) & (possible_probs <= high)]
    # Remove duplicates due to rounding
    possible_probs = np.unique(possible_probs)
    # Ensure there are enough unique probabilities
    if len(possible_probs) < n:
        raise ValueError("Not enough unique probabilities within the specified bounds.")
    for _ in range(1000):  # Maximum attempts
        probs = np.random.choice(possible_probs, size=n, replace=False)
        probs = probs / probs.sum()
        probs = np.round(probs, 2)
        probs[-1] += 1 - probs.sum()  # Adjust to sum to 1
        # Check for uniqueness after rounding and adjustment
        if np.all(probs >= low) and np.all(probs <= high) and len(np.unique(probs)) == n:
            return probs
    raise ValueError("Unable to generate unique probabilities within bounds after rounding.")

# Optimization function
def optimize_lottery(original_probs, original_outcomes):
    # Original lottery statistics
    original_ev = np.sum(original_probs * original_outcomes)
    original_sd = np.sqrt(np.sum(original_probs * (original_outcomes - original_ev) ** 2))
    original_skew = calculate_skewness(original_probs, original_outcomes)

    # Parameters
    n = 7  # Number of outcomes
    iterations = 200  # Number of iterations to run
    delta = 2  # Minimum difference to ensure uniqueness after rounding
    max_gap = 50  # Maximum allowed gap between consecutive outcomes
    gap_weight = 0.01  # Weight for the gap penalty in the objective function

    # Bounds for outcomes
    bounds = [(2, 200)] * n

    # List to store acceptable results
    acceptable_results = []

    # Run the optimization multiple times
    for iteration in range(iterations):
        # Generate unique probabilities
        try:
            fixed_probs = generate_unique_probs(n)
        except ValueError:
            continue  # Skip if unable to generate probabilities

        # Desired average gap between outcomes
        desired_gap = (bounds[0][1] - bounds[0][0]) / (n - 1)

        # Objective function with gap penalty
        def objective_outcomes(outcomes):
            ev, sd, skw = lottery_stats(fixed_probs, outcomes)
            penalty_stats = (ev - original_ev)**2 + (sd - original_sd)**2 + (skw - original_skew)**2
            penalty_gaps = np.sum((np.diff(outcomes) + desired_gap)**2)  # Adjusted for decreasing order
            penalty = penalty_stats + gap_weight * penalty_gaps
            return penalty

        # Initial guess for outcomes (from high to low)
        initial_outcomes = np.linspace(bounds[0][1], bounds[0][0], n)

        # Constraints to ensure outcomes are decreasing and gaps are within limits
        constraints = [
            {'type': 'ineq', 'fun': lambda x, i=i: x[i] - x[i+1] - delta} for i in range(n - 1)
        ] + [
            {'type': 'ineq', 'fun': lambda x, i=i: max_gap - (x[i] - x[i+1])} for i in range(n - 1)
        ]

        # Optimize outcomes
        result = minimize(
            objective_outcomes,
            initial_outcomes,
            bounds=bounds,
            constraints=constraints,
            method='SLSQP',
            options={'ftol': 1e-9, 'disp': False}
        )

        # Check if the optimization was successful
        if not result.success:
            continue  # Skip this iteration if optimization failed

        # Extract optimized outcomes
        optimized_outcomes = np.round(result.x).astype(int)

        # Adjust outcomes to ensure uniqueness after rounding and maintain gaps
        for i in range(1, n):
            # Ensure the gap does not exceed max_gap
            if optimized_outcomes[i - 1] - optimized_outcomes[i] > max_gap:
                optimized_outcomes[i] = optimized_outcomes[i - 1] - max_gap
            # Ensure outcomes are decreasing by at least delta
            if optimized_outcomes[i] >= optimized_outcomes[i - 1]:
                optimized_outcomes[i] = optimized_outcomes[i - 1] - int(delta)
            # Ensure within bounds
            if optimized_outcomes[i] < bounds[i][0]:
                optimized_outcomes[i] = bounds[i][0]

        # Recalculate the statistics with adjusted outcomes
        final_ev, final_sd, final_skew = lottery_stats(fixed_probs, optimized_outcomes)

        # Check if the differences in EV and SD are less than 1
        if abs(final_ev - original_ev) < 1 and abs(final_sd - original_sd) < 1:
            skewness_difference = abs(final_skew - original_skew)
            # Store the acceptable results
            acceptable_results.append({
                'skewness_difference': skewness_difference,
                'outcomes': optimized_outcomes.copy(),
                'probabilities': fixed_probs.copy(),
                'final_ev': final_ev,
                'final_sd': final_sd,
                'final_skew': final_skew,
                'original_ev': original_ev,
                'original_sd': original_sd,
                'original_skew': original_skew
            })

    # After all iterations, select the best result based on minimal skewness difference
    if acceptable_results:
        # Sort the acceptable results by skewness_difference
        acceptable_results.sort(key=lambda x: x['skewness_difference'])
        best_result = acceptable_results[0]
        # Extract the best results
        best_outcomes = best_result['outcomes']
        best_probs = best_result['probabilities']
        final_ev = best_result['final_ev']
        final_sd = best_result['final_sd']
        final_skew = best_result['final_skew']
        # Return the best results and statistics
        return best_outcomes, best_probs, {
            'original_ev': best_result['original_ev'],
            'original_sd': best_result['original_sd'],
            'original_skew': best_result['original_skew'],
            'new_ev': final_ev,
            'new_sd': final_sd,
            'new_skew': final_skew
        }
    else:
        return None, None, None

# Function to process each row of the DataFrame
def process_row(row):
    result_dict = {}
    # For Lottery A
    probs_A = np.array([row['P_A1'], row['P_A2']])
    probs_A = probs_A / probs_A.sum()
    outcomes_A = np.array([row['O_A1'], row['O_A2']])

    # Original statistics for Lottery A
    original_ev_A, original_sd_A, original_skew_A = lottery_stats(probs_A, outcomes_A)
    result_dict['simple_EVA'] = original_ev_A
    result_dict['simple_SDA'] = original_sd_A
    result_dict['simple_skewness_A'] = original_skew_A

    # Optimize Lottery A
    best_outcomes_A, best_probs_A, stats_A = optimize_lottery(probs_A, outcomes_A)

    if best_outcomes_A is not None:
        # Store new outcomes and probabilities
        for i in range(len(best_outcomes_A)):
            result_dict[f'complex_OA{i+1}'] = int(best_outcomes_A[i])  # Ensure integer outcomes
            result_dict[f'complex_PA{i+1}'] = best_probs_A[i]
        # Store new statistics
        result_dict['complex_EVA'] = stats_A['new_ev']
        result_dict['complex_SDA'] = stats_A['new_sd']
        result_dict['complex_skewness_A'] = stats_A['new_skew']
    else:
        for i in range(7):
            result_dict[f'complex_OA{i+1}'] = np.nan
            result_dict[f'complex_PA{i+1}'] = np.nan
        result_dict['complex_EVA'] = np.nan
        result_dict['complex_SDA'] = np.nan
        result_dict['complex_skewness_A'] = np.nan

    # For Lottery B
    probs_B = np.array([row['P_B1'], row['P_B2']])
    probs_B = probs_B / probs_B.sum()
    outcomes_B = np.array([row['O_B1'], row['O_B2']])

    # Original statistics for Lottery B
    original_ev_B, original_sd_B, original_skew_B = lottery_stats(probs_B, outcomes_B)
    result_dict['simple_EVB'] = original_ev_B
    result_dict['simple_SDB'] = original_sd_B
    result_dict['simple_skewness_B'] = original_skew_B

    # Optimize Lottery B
    best_outcomes_B, best_probs_B, stats_B = optimize_lottery(probs_B, outcomes_B)

    if best_outcomes_B is not None:
        # Store new outcomes and probabilities
        for i in range(len(best_outcomes_B)):
            result_dict[f'complex_OB{i+1}'] = int(best_outcomes_B[i])  # Ensure integer outcomes
            result_dict[f'complex_PB{i+1}'] = best_probs_B[i]
        # Store new statistics
        result_dict['complex_EVB'] = stats_B['new_ev']
        result_dict['complex_SDB'] = stats_B['new_sd']
        result_dict['complex_skewness_B'] = stats_B['new_skew']
    else:
        for i in range(7):
            result_dict[f'complex_OB{i+1}'] = np.nan
            result_dict[f'complex_PB{i+1}'] = np.nan
        result_dict['complex_EVB'] = np.nan
        result_dict['complex_SDB'] = np.nan
        result_dict['complex_skewness_B'] = np.nan

    # Differences between Lotteries A and B (Original)
    result_dict['simple_EVD'] = result_dict['simple_EVA'] - result_dict['simple_EVB']
    result_dict['simple_SDD'] = result_dict['simple_SDA'] - result_dict['simple_SDB']
    result_dict['simple_skewness_D'] = result_dict['simple_skewness_A'] - result_dict['simple_skewness_B']

    # Differences between Lotteries A and B (New)
    if best_outcomes_A is not None and best_outcomes_B is not None:
        result_dict['complex_EVD'] = result_dict['complex_EVA'] - result_dict['complex_EVB']
        result_dict['complex_SDD'] = result_dict['complex_SDA'] - result_dict['complex_SDB']
        result_dict['complex_skewness_D'] = result_dict['complex_skewness_A'] - result_dict['complex_skewness_B']
    else:
        result_dict['complex_EVD'] = np.nan
        result_dict['complex_SDD'] = np.nan
        result_dict['complex_skewness_D'] = np.nan

    return pd.Series(result_dict)

# Apply the process_row function to each row
new_columns = final_df_new.apply(process_row, axis=1)

# Concatenate the new columns to the original DataFrame
final_df_new = pd.concat([final_df_new, new_columns], axis=1)

# Convert outcome columns to nullable integer type
outcome_columns = [f'complex_OA{i+1}' for i in range(7)] + [f'complex_OB{i+1}' for i in range(7)]
final_df_new[outcome_columns] = final_df_new[outcome_columns].astype('Int64')



In [28]:

# Ensure the columns are present in the merged_df DataFrame
columns_to_check = ['skew','EVA', 'EVB', 'EVD',
                    'complex_EVA', 'complex_EVB', 'complex_EVD',
                    'SDA', 'SDB', 'SDD',
                    'complex_SDA', 'complex_SDB', 'complex_SDD',
                    'simple_skewness_A', 'complex_skewness_A', 'simple_skewness_B', 'complex_skewness_B',
                    'simple_skewness_D', 'complex_skewness_D'
                    ]

# Check which columns are missing
missing_columns = [col for col in columns_to_check if col not in final_df_new.columns]

# Print missing columns
print(f"Missing columns: {missing_columns}")

# Select only the columns that are present
columns_to_select = [col for col in columns_to_check if col in final_df_new.columns]

df_selected = final_df_new[columns_to_select]

df_selected_rounded = df_selected.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)




df_selected_rounded.sort_values(by='skew', ascending=True)


Missing columns: []


/var/folders/jl/mx4l1z1j1kgd53hpj7770jlw0000gp/T/ipykernel_75417/1049243912.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_selected_rounded = df_selected.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)


,skew,EVA,EVB,EVD,complex_EVA,complex_EVB,complex_EVD,SDA,SDB,SDD,complex_SDA,complex_SDB,complex_SDD,simple_skewness_A,complex_skewness_A,simple_skewness_B,complex_skewness_B,simple_skewness_D,complex_skewness_D
16,lr,82.34,91.96,-9.62,82.15,91.97,-9.82,33.42,18.85,14.58,33.69,19.53,14.16,-1.67,-0.83,2.34,1.73,-4.00,-2.55
31,lr,41.40,31.55,9.85,41.35,31.55,9.80,15.69,5.63,10.06,16.34,6.60,9.75,-1.58,-1.01,1.76,1.78,-3.34,-2.79
21,lr,71.72,71.97,-0.25,71.54,71.81,-0.27,18.04,8.45,9.60,18.59,9.34,9.26,-2.08,-1.56,2.49,2.41,-4.57,-3.97
14,lr,117.66,127.23,-9.57,117.62,127.24,-9.62,33.74,23.88,9.86,34.15,24.51,9.64,-1.58,-1.15,2.20,1.47,-3.78,-2.62
43,lr,68.00,47.42,20.58,67.93,47.34,20.59,24.37,9.77,14.61,25.01,10.53,14.49,-2.34,-1.14,1.76,1.63,-4.10,-2.77
36,lr,83.35,63.36,19.99,83.50,63.64,19.86,16.90,12.25,4.66,17.50,13.01,4.49,-1.76,-1.56,3.37,2.59,-5.13,-4.15
11,lr,51.33,60.88,-9.55,51.43,60.94,-9.51,19.16,14.11,5.05,19.95,14.56,5.39,-1.76,-1.25,4.69,3.12,-6.45,-4.37
8,lr,93.48,113.01,-19.53,93.29,113.06,-19.77,34.98,19.91,15.07,35.10,20.72,14.39,-2.20,-1.07,1.76,1.38,-3.96,-2.45
25,lr,89.33,89.08,0.25,89.46,89.28,0.18,19.84,4.77,15.08,20.62,5.71,14.91,-2.20,-1.70,1.85,1.93,-4.06,-3.63
41,lr,64.82,45.36,19.46,64.59,45.55,19.04,19.59,9.10,10.49,20.20,10.00,10.20,-1.67,-1.24,2.34,2.26,-4.00,-3.50


In [29]:


import math



def Pweight(p, alpha=1, gamma=0.6):
    if p >= 1:
        return 1
    elif p <= 0:
        return 0
    else:
        return math.exp(-alpha * ((-math.log(p)) ** gamma))
    
def apply_probability_weighting(probs):
    weighted_probs = []
    cumulative_prob = 0
    for prob in probs:
        # Since cumulative_prob + prob might exceed 1, ensure it does not
        cumulative_prob_next = min(cumulative_prob + prob, 1)
        weighted_prob = Pweight(cumulative_prob_next) - Pweight(cumulative_prob)
        weighted_probs.append(weighted_prob)
        cumulative_prob = cumulative_prob_next
    return weighted_probs



def calculate_EU(weighted_probs, outcomes):
    return sum(w * o for w, o in zip(weighted_probs, outcomes))

def validate_probabilities(probs):
    # Check if all probabilities are within the valid range (0, 1]
    return all(0 < p <= 1 for p in probs)

def apply_probability_weighting_and_calculate_EUA(row):
    # Extract the probabilities and outcomes from the row
    probs = row[['complex_PA1', 'complex_PA2', 'complex_PA3', 'complex_PA4', 'complex_PA5', 'complex_PA6', 'complex_PA7']].astype(float).tolist()
    outcomes = row[['complex_OA1', 'complex_OA2', 'complex_OA3', 'complex_OA4', 'complex_OA5', 'complex_OA6', 'complex_OA7']].astype(float).tolist()
    
    # Validate probabilities
    if not validate_probabilities(probs):
        # Invalid probabilities detected, return NaN or handle as desired
        return np.nan
    
    # Apply the probability weighting function
    weighted_probs = apply_probability_weighting(probs)
    
    # Calculate the expected utility
    EUA = calculate_EU(weighted_probs, outcomes)
    
    return EUA

def apply_probability_weighting_and_calculate_EUB(row):
    # Extract the probabilities and outcomes from the row
    probs = row[['complex_PB1', 'complex_PB2', 'complex_PB3', 'complex_PB4', 'complex_PB5', 'complex_PB6', 'complex_PB7']].astype(float).tolist()
    outcomes = row[['complex_OB1', 'complex_OB2', 'complex_OB3', 'complex_OB4', 'complex_OB5', 'complex_OB6', 'complex_OB7']].astype(float).tolist()
    
    # Validate probabilities
    if not validate_probabilities(probs):
        # Invalid probabilities detected, return NaN or handle as desired
        return np.nan
    
    # Apply the probability weighting function
    weighted_probs = apply_probability_weighting(probs)
    
    # Calculate the expected utility
    EUB = calculate_EU(weighted_probs, outcomes)
    
    return EUB

def apply_probability_weighting_and_calculate_EUA_simple(row):
    # Extract the probabilities and outcomes from the row
    probs = row[['P_A1', 'P_A2']].astype(float).tolist()
    outcomes = row[['O_A1', 'O_A2']].astype(float).tolist()
    
    # Validate probabilities
    if not validate_probabilities(probs):
        # Invalid probabilities detected, return NaN or handle as desired
        return np.nan
    
    # Apply the probability weighting function
    weighted_probs = apply_probability_weighting(probs)
    
    # Calculate the expected utility
    EUA_simple = calculate_EU(weighted_probs, outcomes)
    
    return EUA_simple

def apply_probability_weighting_and_calculate_EUB_simple(row):
    # Extract the probabilities and outcomes from the row
    probs = row[['P_B1', 'P_B2']].astype(float).tolist()
    outcomes = row[['O_B1', 'O_B2']].astype(float).tolist()
    
    # Validate probabilities
    if not validate_probabilities(probs):
        # Invalid probabilities detected, return NaN or handle as desired
        return np.nan
    
    # Apply the probability weighting function
    weighted_probs = apply_probability_weighting(probs)
    
    # Calculate the expected utility
    EUB_simple = calculate_EU(weighted_probs, outcomes)
    
    return EUB_simple


In [45]:
df_nona = final_df_new.dropna()

# Apply the functions row-wise to compute the new columns
df_nona['EUA'] = df_nona.apply(apply_probability_weighting_and_calculate_EUA, axis=1)
df_nona['EUB'] = df_nona.apply(apply_probability_weighting_and_calculate_EUB, axis=1)
df_nona['EUA_simple'] = df_nona.apply(apply_probability_weighting_and_calculate_EUA_simple, axis=1)
df_nona['EUB_simple'] = df_nona.apply(apply_probability_weighting_and_calculate_EUB_simple, axis=1)

# Remove rows with NaN in EUA or EUB due to invalid probabilities, or handle as desired
df_nona = df_nona.dropna(subset=['EUA', 'EUB', 'EUA_simple', 'EUB_simple'])

# Compute the differences
df_nona['EUD_CC'] = df_nona['EUA'] - df_nona['EUB']
df_nona['EUD_SC'] = df_nona['EUA_simple'] - df_nona['EUB']
df_nona['EUD_CS'] = df_nona['EUA'] - df_nona['EUB_simple']
df_nona['EUD_SS'] = df_nona['EUA_simple'] - df_nona['EUB_simple']

df_nona['after_diff_CC'] = df_nona['EUD_CC'] - df_nona['EVD']
df_nona['after_diff_CS'] = df_nona['EUD_CS'] - df_nona['EVD']
df_nona['after_diff_SC'] = df_nona['EUD_SC'] - df_nona['EVD']
df_nona['after_diff_SS'] = df_nona['EUD_SS'] - df_nona['EVD']




In [46]:

df_nona_sorted = df_nona.sort_values(by='skew', ascending=True)
df_nona_sorted.to_csv('study3_trials_old.csv', index=False)

In [47]:

# Ensure the columns are present in the merged_df DataFrame
columns_to_check = ['skew','EVA', 'EVB', 'EVD',
                    'complex_EVA', 'complex_EVB', 'complex_EVD',
                    'SDA', 'SDB', 'SDD',
                    'complex_SDA', 'complex_SDB', 'complex_SDD',
                    'simple_skewness_A', 'complex_skewness_A', 'simple_skewness_B', 'complex_skewness_B',
                    'simple_skewness_D', 'complex_skewness_D',
                    'after_diff_CC', 'after_diff_CS', 'after_diff_SC', 'after_diff_SS'
                    ]

# Check which columns are missing
missing_columns = [col for col in columns_to_check if col not in df_nona.columns]

# Print missing columns
print(f"Missing columns: {missing_columns}")

# Select only the columns that are present
columns_to_select = [col for col in columns_to_check if col in df_nona.columns]

df_selected = df_nona[columns_to_select]

df_selected_rounded = df_selected.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)




df_selected_rounded_sorted = df_selected_rounded.sort_values(by='skew', ascending=True)

df_selected_rounded_sorted.to_csv('study3_trials_old_short.csv', index=False)


Missing columns: []


/var/folders/jl/mx4l1z1j1kgd53hpj7770jlw0000gp/T/ipykernel_75417/3763124541.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_selected_rounded = df_selected.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)
